
# Source localization with single dipole fit


The aim of this lecture is to show you how to do sequential and
fixed dipole fitting with MNE-Python.

In [1]:
%matplotlib qt
import matplotlib.pyplot as plt

import os
import numpy as np
import mne

mne.set_log_level('WARNING')

# Change the following path to where the folder ds000117-practical is on your disk
data_path = os.path.expanduser("~/work/data/ds000117-practical/")

raw_fname = os.path.join(data_path,
    'derivatives/meg_derivatives/sub-01/ses-meg/meg/sub-01_ses-meg_task-facerecognition_run-01_proc-sss_meg.fif')

evoked_fname = raw_fname.replace('_meg.fif', '-ave.fif')
epochs_fname = raw_fname.replace('_meg.fif', '-epo.fif')

In [2]:
from mne.forward import make_forward_dipole
from mne.evoked import combine_evoked
from mne.simulation import simulate_evoked

subjects_dir = os.path.join(data_path, 'derivatives/freesurfer/sub-01/ses-mri/')
subject = 'anat'
trans_fname = os.path.join(data_path,
    'derivatives/meg_derivatives/sub-01/ses-meg/meg/anat-trans.fif')
bem_fname = os.path.join(data_path,
    'derivatives/meg_derivatives/sub-01/ses-meg/meg/sub-01-bem.fif')
fname_surf_lh = os.path.join(subjects_dir, subject, 'surf', 'lh.white')

Let's localize the N100m (using MEG only):



In [3]:
epochs = mne.read_epochs(epochs_fname)

In [4]:
epochs.pick_types(meg=True, eeg=False)

<EpochsFIF  |   79 events (all good), -0.2 - 2 sec, baseline [-0.2, 0], ~129.1 MB, data loaded,
 'face/famous/first': 13
 'face/famous/immediate': 3
 'face/famous/long': 6
 'face/unfamiliar/first': 17
 'face/unfamiliar/immediate': 4
 'face/unfamiliar/long': 6
 'scrambled/first': 15
 'scrambled/immediate': 9
 'scrambled/long': 6>

In [5]:
cov = mne.compute_covariance(epochs, rank='info')

In [6]:
evoked_face = epochs['face'].average()
evoked_scrambled = epochs['scrambled'].average()

In [7]:
contrast = mne.combine_evoked([evoked_face, evoked_scrambled], [0.5, -0.5])
contrast.crop(None, 0.2)
contrast.plot();

In [8]:
# Fit a dipole using a sequential (time-varying position and orientation) fit
contrast_crop = contrast.copy().crop(0.175, 0.190)
dip, residual = mne.fit_dipole(contrast_crop, cov, bem_fname,
                               trans_fname)
print(dip)

<Dipole  |  n_times : 6, tmin : 0.173, tmax : 0.190>


In [9]:
# Look at our result
print(dip.gof)

[34.0835079  37.06606751 35.91500438 31.77351344 38.54588196 41.60892435]


In [10]:
dip.plot_locations(subject=subject, trans=trans_fname,
                   subjects_dir=subjects_dir, mode='orthoview');

In [11]:
selection = mne.read_selection('Left', info=contrast.info)
# Fit a dipole using a sequential (time-varying position and orientation) fit
dip, residual = \
    mne.fit_dipole(contrast_crop.copy().pick_channels(selection),
                   cov, bem_fname, trans_fname)
print(dip)

<Dipole  |  n_times : 6, tmin : 0.173, tmax : 0.190>


In [12]:
dip.plot_locations(subject=subject, trans=trans_fname,
                   subjects_dir=subjects_dir, mode='orthoview');

## Exercises
- Try to improve the dipole fit by using a subselection of channels:
    - Subselect channels (think about ``mne.read_selection(...)``, ``evoked.pick_channels(...)``)
    - Do a time-varying dipole fit of ``evoked_crop``
    - Do a fixed-position, fixed-orientation fit using ``evoked`` and visualize the result
- What is the improvement in max GOF from using a subselection of channels?
- How far (in mm) is the new best dipole from the original best dipole?